In [ ]:
import os
import re
import cv2
import shutil
import random
import tarfile
import zipfile
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from torchvision.io import read_image
from torchvision import transforms
from pathlib import Path
from os import path

from numpy.random import seed
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.utils import dataset_utils
from keras.preprocessing.image import ImageDataGenerator

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
random_seed = 42

seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

In [ ]:
drive.mount('/content/gdrive', force_remount=True)
with zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/img_align_celeba_for_ncsn.zip","r") as zip_ref:
    zip_ref.extractall()  
with zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/ncsn_generated_samples.zip","r") as zip_ref:
    zip_ref.extractall()

base_dir = "/content/img_align_celeba/"
samples_dir = "/content/samples"

In [ ]:
base_dir_32 = "/content/img_align_celeba_center_32/"

if path.exists(base_dir_32) == False:
  os.mkdir(base_dir_32)

for i in tqdm(range(202599)):
  name = "{0:06d}.jpg".format(i+1)

  img_path = base_dir + name
  img = read_image(img_path)
  img = transforms.ToPILImage()(img)

  # center crop
  width, height = img.size  # Get dimensions
  left = (width - 140) / 2
  top = (height - 140) / 2
  right = (width + 140) / 2
  bottom = (height + 140) / 2
  img = img.crop((left, top, right, bottom))

  # resize
  img = transforms.Resize((32, 32))(img)
  output_path = base_dir_32 + name
  img.save(output_path)

In [ ]:
image_size = (299, 299)

In [ ]:
real_data_dir = pathlib.Path(base_dir_32)
seed_ds = 42 
shuffle_value = False
batch_size = 1

real_ds = tf.keras.preprocessing.image_dataset_from_directory(
    real_data_dir,
    labels=None,
    seed=seed_ds,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=shuffle_value)

In [ ]:
data_dir = pathlib.Path(samples_dir)
seed_ds = 42 
shuffle_value = False
batch_size = 1

data_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels=None,
    seed=seed_ds,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=shuffle_value)

In [ ]:
image.load_img(base_dir_32 + "000001.jpg", target_size=image_size)

In [ ]:
image.load_img(samples_dir + "image_0.png", target_size=image_size)

In [ ]:
def preprocess(images):
  return preprocess_input(images)

real_ds_preprocessed = real_ds.map(preprocess)
data_ds_preprocessed = data_ds.map(preprocess)

# Creazione modello

In [ ]:
model = InceptionV3(weights='imagenet')
model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

feat_extractor_model = Model(inputs=model.input, 
                             outputs=model.get_layer('avg_pool').output)
feat_extractor_model.summary()

In [ ]:
features_real_data = feat_extractor_model.predict(real_ds_preprocessed)
print(features_real_data.shape)

In [ ]:
paths = real_ds.file_paths

In [ ]:
knn = NearestNeighbors(n_neighbors=5, 
                       algorithm='brute',
                       metric='minkowski',
                       p=2)
knn.fit(features_real_data)

# Valutazione soggettiva similarity retrieval per NCSN

In [ ]:
img_path = samples_dir + "image_100.png"
img = image.load_img(img_path, target_size=image_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

im_query = image.load_img(img_path, target_size=image_size)
im_query

In [ ]:
features_new_image = feat_extractor_model.predict(img)
print(features_new_image.shape)

In [ ]:
features_new_image

In [ ]:
# Il primo array indica le distanze
# Il secondo array specifica gli indici delle immagini recuperate
closest_distances, indices = knn.kneighbors(features_new_image, n_neighbors=5)

for i in range(5):
  path_plot = paths[indices[0,i]]
  
  img = image.load_img(path_plot, target_size=image_size)

  plt.figure()
  plt.imshow(img)
  plt.axis('off')